In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np

from lightfm.datasets import fetch_movielens
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import recall_at_k
from lightfm.evaluation import auc_score
from lightfm.data import Dataset

from lightfm.cross_validation import random_train_test_split
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
import itertools

In [3]:
# movielens dataset with 100k movie ratings from 1k users on 1700 movies
data = fetch_movielens(min_rating=2.5)
print(data)

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 74627 stored elements in COOrdinate format>, 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 7893 stored elements in COOrdinate format>, 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
	with 1682 stored elements in Compressed Sparse Row format>, 'item_feature_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
       'Sliding Doors (1998)', 'You So Crazy (1994)',
       'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object), 'item_labels': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
       'Sliding Doors (1998)', 'You So Crazy (1994)',
       'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object)}


In [4]:
train_set = data['train']
test_set = data['test']

In [5]:
# create hybrid model, CB+CF
model = LightFM(learning_rate=0.05,loss='warp')

#train model
model.fit(data['train'],epochs=20,num_threads=2)

In [10]:
result=[]
k = 10
userID_list = [2,10,60]
known_positives =[]
top_movies = []
def lightfm_recommender(model,data,user_ids):
#     # of users and items usinf shape
    no_users,no_movies = data['train'].shape
    
#     generate recommendations for each user we input
    for user_id in user_ids:
#         movies already liked by user so far
        known_positives = data['item_labels'][data['train'].tocsr() [user_id].indices]
#         movies our model predicts they will like
        scores = model.predict(user_id, np.arange(no_movies))
#      rank them in order of most liked to least
        top_movies = data['item_labels'][np.argsort(-scores)]
#       print out the results
        print("User %s" % user_id)
#         userID_list.append(user_id)
        print("     Known positives:")
        
        for x in known_positives[:k]:
            print("        %s" % x)
            
        print("     Recommended:")
        for x in top_movies[:k]:
            print("        %s" % x)
        
        result.append([user_id,known_positives[:k],top_movies[:k]])
        #result_df = pd.DataFrame(result)
        #result_df.rename(columns={0:'User', 1:'known Positives',2:'Top Movies Recommended'},inplace=True)
    return result
    
            
result = lightfm_recommender(model, data, userID_list) 

# result.append([userID_list,known_positives[:k],top_movies[:k]])
result_df = pd.DataFrame(result)
result_df.rename(columns={0:'User', 1:'known Positives',2:'Top Movies Recommended'},inplace=True)


User 2
     Known positives:
        Return of the Jedi (1983)
        Event Horizon (1997)
        Chasing Amy (1997)
        Starship Troopers (1997)
        Hoodlum (1997)
        Ulee's Gold (1997)
        Devil's Advocate, The (1997)
        Schindler's List (1993)
        Paradise Lost: The Child Murders at Robin Hood Hills (1996)
        Mother (1996)
     Recommended:
        Titanic (1997)
        L.A. Confidential (1997)
        Air Force One (1997)
        Contact (1997)
        Cop Land (1997)
        Devil's Advocate, The (1997)
        English Patient, The (1996)
        G.I. Jane (1997)
        Seven Years in Tibet (1997)
        Good Will Hunting (1997)
User 10
     Known positives:
        Babe (1995)
        Dead Man Walking (1995)
        Mr. Holland's Opus (1995)
        Braveheart (1995)
        Rumble in the Bronx (1995)
        Birdcage, The (1996)
        Apollo 13 (1995)
        Batman Forever (1995)
        Strange Days (1995)
        To Wong Foo, Thanks for E

In [22]:
result=[]
known_values =[]
k = 10
userID_list = [2,10,60]
known_positives =[]
top_movies = []
def lightfm_recommender(model,data,user_ids):
#     # of users and items usinf shape
    no_users,no_movies = data['train'].shape
    
#     generate recommendations for each user we input
    for user_id in user_ids:
#         movies already liked by user so far
        known_positives = data['item_labels'][data['train'].tocsr() [user_id].indices]
#         movies our model predicts they will like
        scores = model.predict(user_id, np.arange(no_movies))
#      rank them in order of most liked to least
        top_movies = data['item_labels'][np.argsort(-scores)]
#       print out the results
        print("User %s" % user_id)
#         userID_list.append(user_id)
        print("     Known positives:")
        
        for x in known_positives[:k]:
            print("        %s" % x)
            known_values.append([user_id,x])
            
        print("     Recommended:")
        for x in top_movies[:k]:
            print("        %s" % x)
            result.append([user_id,x])
    return known_values,result
             
known_values, result = lightfm_recommender(model, data, userID_list)

known_values_df = pd.DataFrame(known_values)
known_values_df.rename(columns={0:'User',1:'Known Positives'},inplace=True)
result_df = pd.DataFrame(result)
result_df.rename(columns={0:'User',1:'Top Movies Recommended'},inplace=True)


User 2
     Known positives:
        Return of the Jedi (1983)
        Event Horizon (1997)
        Chasing Amy (1997)
        Starship Troopers (1997)
        Hoodlum (1997)
        Ulee's Gold (1997)
        Devil's Advocate, The (1997)
        Schindler's List (1993)
        Paradise Lost: The Child Murders at Robin Hood Hills (1996)
        Mother (1996)
     Recommended:
        Titanic (1997)
        L.A. Confidential (1997)
        Air Force One (1997)
        Contact (1997)
        Cop Land (1997)
        Devil's Advocate, The (1997)
        English Patient, The (1996)
        G.I. Jane (1997)
        Seven Years in Tibet (1997)
        Good Will Hunting (1997)
User 10
     Known positives:
        Babe (1995)
        Dead Man Walking (1995)
        Mr. Holland's Opus (1995)
        Braveheart (1995)
        Rumble in the Bronx (1995)
        Birdcage, The (1996)
        Apollo 13 (1995)
        Batman Forever (1995)
        Strange Days (1995)
        To Wong Foo, Thanks for E

In [23]:
result_df

,User,Top Movies Recommended
0,2,Titanic (1997)
1,2,L.A. Confidential (1997)
2,2,Air Force One (1997)
3,2,Contact (1997)
4,2,Cop Land (1997)
5,2,"Devil's Advocate, The (1997)"
6,2,"English Patient, The (1996)"
7,2,G.I. Jane (1997)
8,2,Seven Years in Tibet (1997)
9,2,Good Will Hunting (1997)


In [24]:
known_values_df

,User,Known Positives
0,2,Return of the Jedi (1983)
1,2,Event Horizon (1997)
2,2,Chasing Amy (1997)
3,2,Starship Troopers (1997)
4,2,Hoodlum (1997)
5,2,Ulee's Gold (1997)
6,2,"Devil's Advocate, The (1997)"
7,2,Schindler's List (1993)
8,2,Paradise Lost: The Child Murders at Robin Hood...
9,2,Mother (1996)


In [ ]:
# Evaluation
from lightfm.evaluation import precision_at_k
train_p_at_k = precision_at_k(model, data['train'], k=5).mean()
test_p_at_k = precision_at_k(model, data['test'], k=5).mean()
print("PRECISION@K: Train precision: %.4f" % train_p_at_k)
print("PRECISION@K: Test precision: %.4f" % test_p_at_k)

In [ ]:
train_auc = auc_score(model, train_set).mean()
test_auc = auc_score(model, test_set).mean()
print('AUC: train %.4f, test %.4f.' % (train_auc, test_auc))

In [ ]:
train_recall = recall_at_k(model, train_set).mean()
test_recall = recall_at_k(model, test_set).mean()
print('RECALL@K: train %.4f, test %.4f.' % (train_recall, test_recall))

In [ ]:
dataset = Dataset()
# dataset.fit(data['item_features'],data['item_feature_labels'])
movieXuser = ratings_df.pivot(index='movieId',columns='userId',values='rating').fillna(0)
# print(movieXuser)
# convert into Sparse matrix
MxU_mat = csr_matrix(movieXuser.values)
# print(data['item_features']) #sparse matrix
# (interactions, weights) = dataset.build_interactions(data['item_features'])

(train_interactions, test_interactions) = random_train_test_split(interactions=data['item_features'], test_percentage=0.2)
train_interactions, test_interactions = train_test_split(data['item_features'], test_size = 0.2, random_state=20)

print(train_interactions.shape)
print(test_interactions.shape)
print(train_set.shape)
print(test_set.shape)

Evaluation using hyper parameters

AUC Score:

In [ ]:
def sample_hyperparameters():
    """
    Yield possible hyperparameter choices.
    """

    while True:
        yield {
            "no_components": np.random.randint(16, 64),
            "learning_schedule": np.random.choice(["adagrad", "adadelta"]),
            "loss": np.random.choice(["bpr", "warp", "warp-kos"]),
            "learning_rate": np.random.exponential(0.05),
            "item_alpha": np.random.exponential(1e-8),
            "user_alpha": np.random.exponential(1e-8),
            "max_sampled": np.random.randint(5, 15),
            "num_epochs": np.random.randint(5, 50),
            
        }


def random_search_auc(train, test, num_samples=10, num_threads=1,k=5):

    for hyperparams in itertools.islice(sample_hyperparameters(), num_samples):
        num_epochs = hyperparams.pop("num_epochs")
        model = LightFM(**hyperparams)
        model.fit(train, epochs=num_epochs, num_threads=num_threads)
        score = auc_score(model, test, train_interactions=train, num_threads=num_threads,check_intersections=False).mean()
        hyperparams["num_epochs"] = num_epochs
        yield (score, hyperparams, model)


if __name__ == "__main__":
    from lightfm.datasets import fetch_movielens
    data = fetch_movielens()
    train = data["train"]
    test = data["test"]
    (score, hyperparams, model) = max(random_search_auc(train_set, test_set, num_threads=2), key=lambda x: x[0])

    print("Best AUC score {} at {}".format(score, hyperparams))

    

Precision@K:

In [ ]:
def random_search_precision_at_k(train, test, num_samples=5, num_threads=2,k=5):
    
    for hyperparams in itertools.islice(sample_hyperparameters(), num_samples):
        num_epochs = hyperparams.pop("num_epochs")

        model = LightFM(**hyperparams)
        model.fit(train, epochs=num_epochs, num_threads=num_threads)

        p_at_k_score = precision_at_k(model, test_interactions=test, train_interactions=train, num_threads=num_threads,k=k,check_intersections=False,preserve_rows=True).mean()
        
        hyperparams["num_epochs"] = num_epochs

        yield (p_at_k_score, hyperparams, model)


if __name__ == "__main__":
    from lightfm.datasets import fetch_movielens

    data = fetch_movielens()
    train = data["train"]
    test = data["test"]

    (p_at_k_score, hyperparams, model) = max(random_search_precision_at_k(train_set, test_set, num_threads=2,k=5), key=lambda x: x[0])

    print("Best Precision@k score {} at {}".format(p_at_k_score, hyperparams))


Recall@K

In [ ]:
def random_search_recall_at_k(train, test, num_samples=10, num_threads=1,k=5):
    

    for hyperparams in itertools.islice(sample_hyperparameters(), num_samples):
        num_epochs = hyperparams.pop("num_epochs")

        model = LightFM(**hyperparams)
        model.fit(train, epochs=num_epochs, num_threads=num_threads)

        r_at_k_score = recall_at_k(model, test, train_interactions=train, num_threads=num_threads, k=k).mean()
        
        hyperparams["num_epochs"] = num_epochs

        yield (r_at_k_score, hyperparams, model)


if __name__ == "__main__":
    from lightfm.datasets import fetch_movielens

    data = fetch_movielens()
    train = data["train"]
    test = data["test"]

    (r_at_k_score, hyperparams, model) = max(random_search_recall_at_k(train_set, test_set, num_threads=2), key=lambda x: x[0])

    print("Best Recall@k score {} at {}".format(r_at_k_score, hyperparams))


Reference: The above program is referred from the documentation of existing hybrod model.